In [2]:
#step 1:load the order.csv
import pandas as pd
orders=pd.read_csv("orders.csv")

In [3]:
#step 2:load the user.json
users=pd.read_json("users.json")

In [4]:
#step 3:load restaurants.sql
import sqlite3
conn=sqlite3.connect("restaurants.db")
cursor=conn.cursor()
with open("restaurants.sql","r") as f:
  sql_script=f.read()
cursor.executescript(sql_script)
restaurants=pd.read_sql_query("SELECT * FROM restaurants",conn)

In [5]:
#step 4:merge orders+users
merged_1=pd.merge(orders,users,on="user_id",how="left")

In [6]:
final_df=pd.merge(merged_1,restaurants,on="restaurant_id",how="left")

In [7]:
#step 5:save final dataset
final_df.to_csv("final_food_delivery_dataset.csv",index=False)

##Analysis


In [8]:
#Q1.
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"].sum().sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [9]:
#Q2.
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [10]:
#Q3.
final_df.groupby("user_id")["total_amount"].sum().gt(1000).sum()

np.int64(2544)

In [29]:
#Q4.
final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=[0,3.5,4.0,4.5,5.0],
    labels=["3.0–3.5","3.6–4.0","4.1–4.5","4.6–5.0"])

final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


/tmp/ipython-input-1650858360.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


,total_amount
rating_range,
4.6–5.0,2197030.75
3.0–3.5,2136772.70
4.1–4.5,1960326.26
3.6–4.0,1717494.41


In [14]:
#Q5.
final_df[final_df["membership"]=="Gold"] \
.groupby("city")["total_amount"].mean().sort_values(ascending=False)


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [24]:
#Q6.
final_df.groupby("cuisine").agg(revenue=("total_amount","sum"),restaurants=("restaurant_id","nunique"))


,revenue,restaurants
cuisine,,
Chinese,1930504.65,120
Indian,1971412.58,126
Italian,2024203.80,126
Mexican,2085503.09,128


In [25]:
#Q7.
(final_df["membership"].value_counts(normalize=True)*100).round()

,proportion
membership,
Regular,50.0
Gold,50.0


In [15]:
final_df.head()
final_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [16]:
final_df[final_df["membership"]=="Gold"].shape[0]


4987

In [17]:
round(final_df[final_df["city"]=="Hyderabad"]["total_amount"].sum())


1889367

In [18]:
final_df["user_id"].nunique()


2883

In [19]:
round(final_df[final_df["membership"]=="Gold"]["total_amount"].mean(),2)


np.float64(797.15)

In [21]:
gold_df = final_df[final_df["membership"]=="Gold"]
top_city = gold_df.groupby("city")["total_amount"].sum().idxmax()

gold_df[gold_df["city"] == top_city].shape[0]

1337